In [1]:
!pip install pytorch-transformers
!git clone https://github.com/HyunjoonCho/CS492I-IntroToDL-project.git
import os
os.chdir('CS492I-IntroToDL-project')

     |████████████████████████████████| 176 kB 5.5 MB/s 
     |████████████████████████████████| 131 kB 37.7 MB/s 
     |████████████████████████████████| 1.2 MB 41.6 MB/s 
     |████████████████████████████████| 895 kB 31.0 MB/s 
     |████████████████████████████████| 8.4 MB 33.3 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 138 kB 45.1 MB/s 
     |████████████████████████████████| 127 kB 29.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Cloning into 'CS492I-IntroToDL-project'...
remote: Enumerating objects: 23179, done.
remote: Counting objects: 10

In [2]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from pathlib import Path

import os

In [3]:
from google.colab import drive
drive.mount('/gdrive')

drive_root = '/gdrive/My Drive/CS492I/project-pretrain'
print(os.listdir(Path(drive_root)))

Mounted at /gdrive
['BertForSequenceClassification_Category_best.ckpt', 'BertForSequenceClassification_Fluctuation_best.ckpt']


In [4]:
from easydict import EasyDict as edict

args = edict()
args.gpu = True
args.batch_size = 4
args.num_epochs = 15
args.learning_rate = 1e-6

device = 'cuda' if torch.cuda.is_available() and args.gpu else 'cpu'

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_category_clf = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=8)

100%|██████████| 714314041/714314041 [00:21<00:00, 32774785.02B/s]


In [6]:
dataset_train = []
dataset_val = []
dataset_test = []

root = Path('dataset/category')
list = os.listdir(root)
for cat in list:
    files = os.listdir(root / cat)
    for i,f in enumerate(files):
        fname = root / cat / f
        with open(fname, "r", encoding="utf-8") as file:
            strings = file.read()
            if i < 150:
                dataset_train.append([strings, cat])
            elif i < 175:
                dataset_val.append([strings, cat])
            else:
                dataset_test.append([strings,cat])

print(len(dataset_train), len(dataset_val), len(dataset_test))

1200 200 200


In [7]:
print(dataset_train[0][0][:64]) #sentence
print(dataset_train[0][1]) #label

데이터만 사용하는 알뜰폰 나온다	음성·문자없는 ‘데이터온리’ 상품
6월 알뜰폰 활성화 방안에 포함
가격경쟁력 대신 
5


In [8]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx):
        # 현재 i[sent_idx] 가 본문
        self.sentences = [i[sent_idx][:64] for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return self.sentences[i], self.labels[i]

    def __len__(self):
        return (len(self.labels))

In [9]:
data_train = BERTDataset(dataset_train, 0, 1)
data_val = BERTDataset(dataset_val, 0, 1)
data_test = BERTDataset(dataset_test, 0, 1)

In [10]:
train_dataloader = DataLoader(data_train, batch_size=args.batch_size, num_workers=5, shuffle=True)
val_dataloader = DataLoader(data_val, batch_size=args.batch_size, num_workers=5, shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=args.batch_size, num_workers=5, shuffle=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [11]:
bert_category_clf.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [12]:
def save_model(model, mode='last'):
    torch.save(model.state_dict(),  Path('pretrained_models') / f'{type(model).__name__}_Category_{mode}.ckpt')
    torch.save(model.state_dict(), Path(drive_root) / f'{type(model).__name__}_Category_{mode}.ckpt')

In [13]:
#training step
optimizer = optim.AdamW(bert_category_clf.parameters(), lr=1e-6)
best_val_loss = float('inf')

for epoch in range(args.num_epochs):
    train_loss = 0
    total_len = 0
    total_correct = 0
    bert_category_clf.train()
    for sentence, label in train_dataloader:
        optimizer.zero_grad()
        
        encoded_list = [tokenizer.encode(t,add_special_tokens=True) for t in sentence]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        label = tuple((int(x[0])) for x in label)
        label = torch.tensor(label)
        sample = sample.to(device)
        label = label.to(device)
        
        labels = torch.tensor(label)
        loss, logits = bert_category_clf(sample, labels=labels)

        pred = torch.argmax(F.softmax(logits), dim=1)        
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        train_loss += loss.item()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        #scheduler.step()        
    print('[Epoch {}/{}] -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, args.num_epochs, train_loss, total_correct/total_len))

    with torch.no_grad():
        bert_category_clf.eval()
        val_loss = 0
        v_total_correct = 0
        v_total_len = 0
        for sentence, label in val_dataloader:
            encoded_list = [tokenizer.encode(t,add_special_tokens=True) for t in sentence]
            padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
            
            sample = torch.tensor(padded_list)
            label = tuple((int(x[0])) for x in label)
            label = torch.tensor(label)
            sample = sample.to(device)
            label = label.to(device)
            
            labels = torch.tensor(label)
            loss, logits = bert_category_clf(sample, labels=labels)
            
            pred = torch.argmax(F.softmax(logits), dim=1)        
            correct = pred.eq(labels)
            val_loss += loss.item()
            v_total_correct += correct.sum().item()
            v_total_len += len(labels)
        print('[Epoch {}/{}] -> Validation Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch + 1, args.num_epochs, val_loss, v_total_correct / v_total_len))
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model(bert_category_clf, 'best')
    save_model(bert_category_clf)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/15] -> Train Loss: 626.4617, Accuracy: 0.126


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/15] -> Validation Loss: 103.6096, Accuracy: 0.135
[Epoch 2/15] -> Train Loss: 615.9930, Accuracy: 0.186
[Epoch 2/15] -> Validation Loss: 102.0773, Accuracy: 0.155
[Epoch 3/15] -> Train Loss: 583.0256, Accuracy: 0.294
[Epoch 3/15] -> Validation Loss: 91.3869, Accuracy: 0.475
[Epoch 4/15] -> Train Loss: 520.7862, Accuracy: 0.468
[Epoch 4/15] -> Validation Loss: 79.6258, Accuracy: 0.570
[Epoch 5/15] -> Train Loss: 442.3048, Accuracy: 0.573
[Epoch 5/15] -> Validation Loss: 69.2369, Accuracy: 0.585
[Epoch 6/15] -> Train Loss: 380.9533, Accuracy: 0.658
[Epoch 6/15] -> Validation Loss: 65.4018, Accuracy: 0.620
[Epoch 7/15] -> Train Loss: 333.5658, Accuracy: 0.699
[Epoch 7/15] -> Validation Loss: 57.7941, Accuracy: 0.670
[Epoch 8/15] -> Train Loss: 298.3588, Accuracy: 0.742
[Epoch 8/15] -> Validation Loss: 54.5529, Accuracy: 0.675
[Epoch 9/15] -> Train Loss: 270.4927, Accuracy: 0.770
[Epoch 9/15] -> Validation Loss: 50.2049, Accuracy: 0.675
[Epoch 10/15] -> Train Loss: 243.3529, Accur

In [14]:
bert_category_clf.eval()

t_total_len = 0
t_total_correct = 0
with torch.no_grad():
    for sentence, label in test_dataloader:
        encoded_list = [tokenizer.encode(t,add_special_tokens=True) for t in sentence]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        sample = torch.tensor(padded_list)
        label = tuple((int(x[0])) for x in label)
        label = torch.tensor(label)
        sample = sample.to(device)
        label = label.to(device)
            
        labels = torch.tensor(label)
        _, logits = bert_category_clf(sample, labels=labels)

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        t_total_correct += correct.sum().item()
        t_total_len += len(labels)

print('Test accuracy: ', t_total_correct / t_total_len)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test accuracy:  0.755


In [15]:
def test_model(model, seq):
    cate = ["정치","경제","사회", "생활/문화","세계","기술/IT", "연예", "스포츠"]
    tmp = [seq]
    encoded_list = [tokenizer.encode(t,add_special_tokens=True) for t in tmp]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)

    labels = torch.tensor([1]).unsqueeze(0)
    sample = sample.to(device)
    labels = labels.to(device)
    _, logits = model(sample, labels=labels)

    pred = torch.argmax(F.softmax(logits), dim=1)

    print("뉴스의 카테고리는:", cate[pred])
    print("신뢰도는:", "{:.2f}%".format(F.softmax(logits).max().item() * 100))

In [20]:
test_model(bert_category_clf, "'승진 끝판왕' 삼성전자 대표이사급 연봉 어느 정도길래?")

뉴스의 카테고리는: 경제
신뢰도는: 53.92%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [22]:
bert_category_clf2 = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=8)
bert_category_clf2.load_state_dict(torch.load(Path(drive_root) / f'{type(bert_category_clf2).__name__}_Category_best.ckpt', map_location=device))

<All keys matched successfully>

In [24]:
bert_category_clf2.to(device)
test_model(bert_category_clf2, "'승진 끝판왕' 삼성전자 대표이사급 연봉 어느 정도길래?")

뉴스의 카테고리는: 경제
신뢰도는: 53.92%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
